In [49]:
fips_file = './fips.txt'

In [93]:
import pandas as pd
import geopandas
import json

In [51]:
# get full data
with open(fips_file, 'r') as f:
    text = f.read()

In [52]:
text_top = text.split('state-level')[0]
states_text = text.split('state-level')[-1].split('county-level')[0]
counties_text = text.split('state-level')[-1].split('county-level')[-1]

# more cleaning
states_text = states_text.split('-----------')[-1].split(' -------')[-1]
counties_text = counties_text.split('--------------')[-1]

In [53]:
#print(counties_text)

## US States

In [54]:
# clean states
print(states_text)


       01        ALABAMA
       02        ALASKA
       04        ARIZONA
       05        ARKANSAS
       06        CALIFORNIA
       08        COLORADO
       09        CONNECTICUT
       10        DELAWARE
       11        DISTRICT OF COLUMBIA
       12        FLORIDA
       13        GEORGIA
       15        HAWAII
       16        IDAHO
       17        ILLINOIS
       18        INDIANA
       19        IOWA
       20        KANSAS
       21        KENTUCKY
       22        LOUISIANA
       23        MAINE
       24        MARYLAND
       25        MASSACHUSETTS
       26        MICHIGAN
       27        MINNESOTA
       28        MISSISSIPPI
       29        MISSOURI
       30        MONTANA
       31        NEBRASKA
       32        NEVADA
       33        NEW HAMPSHIRE
       34        NEW JERSEY
       35        NEW MEXICO
       36        NEW YORK
       37        NORTH CAROLINA
       38        NORTH DAKOTA
       39        OHIO
       40        OKLAHOMA
       41        OR

In [55]:
states = {'id':[], 'state':[]}

for l in states_text.split('\n'):
    ll = l.split('      ')
    for lll in ll:
        if len(lll.strip()) > 0:
            if lll.strip().isdigit():
                states['id'].append(lll.strip())
            else:
                states['state'].append(lll.strip())


In [56]:
states_df = pd.DataFrame(states)

In [57]:
states_df

,id,state
0,01,ALABAMA
1,02,ALASKA
2,04,ARIZONA
3,05,ARKANSAS
4,06,CALIFORNIA
5,08,COLORADO
6,09,CONNECTICUT
7,10,DELAWARE
8,11,DISTRICT OF COLUMBIA
9,12,FLORIDA


In [58]:
# save dataframe
states_df.to_csv('./us_state_codes.csv',index=False)

### Putting things together with geospatial info

In [64]:
from vega_datasets import data

In [85]:
states_geo_in = geopandas.read_file(data.us_10m.url,layer='states')

In [86]:
state_names = []
for i in states_geo_in['id'].values:
    d = states_df[states_df['id']==str(i).zfill(2)]
    if len(d) != 1:
        if int(i) == 72:
            n = 'Puerto Rico'
            state_names.append(n)
        elif int(i) == 78:
            state_names.append('U.S. Virgin Islands')
        else:
            print('issue with not having the right name! ID=',i)
        #import sys; sys.exit()
    else:
        n = d['state'].values[0]
        state_names.append(n)
    #print(n)

states_geo = states_geo_in.copy()
states_geo['State'] = state_names


In [106]:
#states_geo

In [ ]:
# save to file
with open('./us_states_geo.geojson','w') as f:
    print(states_geo.to_json(),file=f)

In [103]:
states_geo_in = geopandas.read_file('./us_states_geo.geojson')

In [105]:
#

## US Counties

In [59]:
counties = {'id':[], 'county':[]}

for l in counties_text.split('\n'):
    ll = l.split('    ')
    for lll in ll:
        if len(lll.strip()) > 0:
            if lll.strip().isdigit():
                counties['id'].append(lll.strip())
            else:
                if '(' not in lll:
                    c = lll.strip()
                    counties['county'].append(c)
                else:
                    c = lll.split('(')[0] # take out any notes
                    c = c.strip()
                    # > 0?
                    if len(c) > 0:
                        counties['county'].append(c)
    # check
    if len(counties['id']) != len(counties['county']):
        print('mis match length')
        import sys; sys.exit()


In [60]:
#counties['id'][-5:], counties['county'][-5:]

In [61]:
counties_df = pd.DataFrame(counties)

In [62]:
counties_df

,id,county
0,01000,Alabama
1,01001,Autauga County
2,01003,Baldwin County
3,01005,Barbour County
4,01007,Bibb County
...,...,...
3190,56037,Sweetwater County
3191,56039,Teton County
3192,56041,Uinta County
3193,56043,Washakie County


In [63]:
# save dataframe
counties_df.to_csv('./us_county_codes.csv',index=False)

In [107]:
# also with geospatial stuff
counties_geo_in = geopandas.read_file(data.us_10m.url,layer='counties')

In [112]:
counties_geo_in['id']

0       22051
1       53000
2       53073
3       30105
4       30029
        ...  
3226    72037
3227    72069
3228    72147
3229    78010
3230    72051
Name: id, Length: 3231, dtype: object

In [113]:
counties_df['id']

0       01000
1       01001
2       01003
3       01005
4       01007
        ...  
3190    56037
3191    56039
3192    56041
3193    56043
3194    56045
Name: id, Length: 3195, dtype: object

In [139]:
county_names = []
for i in counties_geo_in['id'].values:
    d = counties_df[counties_df['id'].astype('int')==int(i)]
    if len(d) != 1:
        if int(i) == 8014:
            county_names.append('Broomfield county')
        elif int(i) == 12086:
            county_names.append('Miami-Dade County')
        # elif int(i) == 72125:
        #     county_names.append('San German Municipio')
        # elif int(i) == 72003:
        #     county_names.append('Aguada Municipio')
        else:
            print('issue with not having the right name! ID=',i)
            #import sys; sys.exit()
            # ignore
            county_names.append('NaN')
    else:
        n = d['county'].values[0]
        county_names.append(n)
    #print(n)

counties_geo = counties_geo_in.copy()
counties_geo['County'] = county_names

# subset
counties_geo = counties_geo[counties_geo['County']!='NaN']

issue with not having the right name! ID= 72125
issue with not having the right name! ID= 72003
issue with not having the right name! ID= 72097
issue with not having the right name! ID= 72065
issue with not having the right name! ID= 72055
issue with not having the right name! ID= 72083
issue with not having the right name! ID= 72025
issue with not having the right name! ID= 72045
issue with not having the right name! ID= 72133
issue with not having the right name! ID= 72121
issue with not having the right name! ID= 72027
issue with not having the right name! ID= 72001
issue with not having the right name! ID= 72111
issue with not having the right name! ID= 72047
issue with not having the right name! ID= 72091
issue with not having the right name! ID= 72013
issue with not having the right name! ID= 72145
issue with not having the right name! ID= 72031
issue with not having the right name! ID= 72061
issue with not having the right name! ID= 72129
issue with not having the right name! ID

In [140]:
counties_geo

,id,geometry,County
0,22051,MULTIPOLYGON EMPTY,Jefferson Parish
1,53000,"POLYGON ((-122.65544 48.41032, -122.65544 48.4...",Washington
2,53073,"MULTIPOLYGON (((-120.85361 49.00011, -120.7674...",Whatcom County
3,30105,"POLYGON ((-106.11238 48.99904, -106.15187 48.8...",Valley County
4,30029,"POLYGON ((-114.06985 48.99904, -114.05908 48.8...",Flathead County
...,...,...,...
3149,2201,"POLYGON ((-133.52741 55.6952, -133.69611 55.78...",Prince of Wales-Outer Ketchikan Census Area
3150,2201,"POLYGON ((-133.25822 55.77416, -133.34436 55.8...",Prince of Wales-Outer Ketchikan Census Area
3151,2201,"POLYGON ((-133.72123 55.90146, -133.81814 55.9...",Prince of Wales-Outer Ketchikan Census Area
3152,2201,"POLYGON ((-132.928 56.0148, -132.928 56.01749,...",Prince of Wales-Outer Ketchikan Census Area


In [130]:
d

,id,county


In [116]:
counties_geo

,id,geometry
0,22051,MULTIPOLYGON EMPTY
1,53000,"POLYGON ((-122.65544 48.41032, -122.65544 48.4..."
2,53073,"MULTIPOLYGON (((-120.85361 49.00011, -120.7674..."
3,30105,"POLYGON ((-106.11238 48.99904, -106.15187 48.8..."
4,30029,"POLYGON ((-114.06985 48.99904, -114.05908 48.8..."
...,...,...
3226,72037,"MULTIPOLYGON (((-65.63591 18.28406, -65.63591 ..."
3227,72069,"POLYGON ((-65.74 18.17395, -65.79743 18.06974,..."
3228,72147,"POLYGON ((-65.53182 18.07995, -65.57849 18.118..."
3229,78010,"POLYGON ((-64.90011 17.6744, -64.87139 17.7705..."


In [141]:
# save to file
with open('./us_counties_geo.geojson','w') as f:
    print(counties_geo.to_json(),file=f)